In [1]:
from utils.imports import *

Using TensorFlow backend.


In [2]:
def load_train():
    folders = [x for x in os.listdir(data_path) if 'subset' in x]
    os.chdir(data_path)
    patients = []    
    for i in folders:
        os.chdir(data_path + i)
        patient_ids = [x for x in os.listdir(data_path + i) if '.mhd' in x]
        for id in patient_ids:
            j = '{}/{}'.format(i, id)
            patients.append(j)
    return patients

def get_filename(file_list, case):
    for f in file_list:
        if case in f:
            return(f)

def normalize(image):
    MIN_BOUND = -1000.0
    MAX_BOUND = 400.0
    image = (image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    image[image > 1] = 1.
    image[image < 0] = 0.
    return image

def save_cube_img(target_path, cube_img, rows, cols):
    assert rows * cols == cube_img.shape[0]
    img_height = cube_img.shape[1]
    img_width = cube_img.shape[1]
    res_img = numpy.zeros((rows * img_height, cols * img_width), dtype=numpy.uint8)

    for row in range(rows):
        for col in range(cols):
            target_y = row * img_height
            target_x = col * img_width
            res_img[target_y:target_y + img_height, target_x:target_x + img_width] = cube_img[row * cols + col]

    cv2.imwrite(target_path, res_img)


def get_cube_from_img(img3d, center_x, center_y, center_z, block_size):
    start_x = max(center_x - block_size / 2, 0)
    if start_x + block_size > img3d.shape[2]:
        start_x = img3d.shape[2] - block_size

    start_y = max(center_y - block_size / 2, 0)
    start_z = max(center_z - block_size / 2, 0)
    if start_z + block_size > img3d.shape[0]:
        start_z = img3d.shape[0] - block_size
    start_z = int(start_z)
    start_y = int(start_y)
    start_x = int(start_x)
    res = img3d[start_z:start_z + block_size, start_y:start_y + block_size, start_x:start_x + block_size]
    return res

def create_samples(img_file): 

    
    #1、生成肺部及肺部掩模图片
    patient_id = img_file.split('/')[-1][:-4]
    dst_dir = pic_path + patient_id + "/"
    itk_img = SimpleITK.ReadImage(data_path + img_file)
    img_array = SimpleITK.GetArrayFromImage(itk_img)
    origin = numpy.array(itk_img.GetOrigin())      # x,y,z  Origin in world coordinates (mm)
    direction = numpy.array(itk_img.GetDirection())      # x,y,z  Origin in world coordinates (mm)
    spacing = numpy.array(itk_img.GetSpacing())    # spacing of voxels in world coor. (mm)
    rescale = spacing / TARGET_VOXEL_MM
    #img_array = helpers.rescale_patient_images(img_array, spacing, TARGET_VOXEL_MM)
    df_node = pandas.read_csv(csv_path + "annotations.csv")
    df_patient = df_node[df_node["seriesuid"] == patient_id]

    flip_direction_x = False
    flip_direction_y = False
    if round(direction[0]) == -1:
        origin[0] *= -1
        direction[0] = 1
        flip_direction_x = True
    if round(direction[4]) == -1:
        origin[1] *= -1
        direction[4] = 1
        flip_direction_y = True
    assert abs(sum(direction) - 3) < 0.01
    patient_imgs = helpers.load_patient_images(patient_id, pic_path, "*_i.png")
    pos_annos = []
    anno_index = 0
    for index, annotation in df_patient.iterrows():
        node_x = annotation["coordX"]
        if flip_direction_x:
            node_x *= -1
        node_y = annotation["coordY"]
        if flip_direction_y:
            node_y *= -1
        node_z = annotation["coordZ"]
        diam_mm = annotation["diameter_mm"]
        center_float = numpy.array([node_x, node_y, node_z])  #world
        center_int = numpy.rint((center_float-origin) / spacing) #voxel
        
        coord_x = center_float[2]
        coord_y = center_float[1]
        coord_z = center_float[0]
        radius = np.ceil(diam_mm/2)
        noduleRange = seq(-radius, radius, 1)
        image_mask = np.zeros(patient_imgs.shape)

        for x in noduleRange:
            for y in noduleRange:
                for z in noduleRange:
                    coords = numpy.rint((np.array((coord_z+z,coord_y+y,coord_x+x))-origin) / spacing)
                    if (np.linalg.norm(voxel_center - coords)) < radius:
                        image_mask[int(np.round(coords[0])),int(np.round(coords[1])),int(np.round(coords[2]))] = int(1)
        #print(np.max(image_mask))            
    
    img_list = []
    for i in range(img_array.shape[0]):
        img = img_array[i]
        seg_img, mask = helpers.get_segmented_lungs(img.copy())
        img_list.append(seg_img)
        img = normalize(img)
        cv2.imwrite(dst_dir + "img_" + str(i).rjust(4, '0') + "_i.png", img * 255)
        cv2.imwrite(dst_dir + "img_" + str(i).rjust(4, '0') + "_m.png", mask * 255)
        cv2.imwrite(dst_dir + "img_" + str(i).rjust(4, '0') + "_o.png", image_mask[i])
    return


def create_tests(img_file): 
    #1、生成肺部及肺部掩模图片
    patient_id = img_file.split('/')[-1][:-4]
    dst_dir = pic_path + patient_id + "/"
    itk_img = SimpleITK.ReadImage(data_path + img_file)
    img_array = SimpleITK.GetArrayFromImage(itk_img)
    origin = numpy.array(itk_img.GetOrigin())      # x,y,z  Origin in world coordinates (mm)
    direction = numpy.array(itk_img.GetDirection())      # x,y,z  Origin in world coordinates (mm)
    spacing = numpy.array(itk_img.GetSpacing())    # spacing of voxels in world coor. (mm)
    rescale = spacing / TARGET_VOXEL_MM
    #img_array = helpers.rescale_patient_images(img_array, spacing, TARGET_VOXEL_MM)
    
    img_list = []
    for i in range(img_array.shape[0]):
        img = img_array[i]
        seg_img, mask = helpers.get_segmented_lungs(img.copy())
        img_list.append(seg_img)
        img = normalize(img)
        if not os.path.exists(dst_dir):
            os.mkdir(dst_dir)
        cv2.imwrite(dst_dir + "img_" + str(i).rjust(4, '0') + "_i.png", img * mask * 255)

## Train

In [ ]:
data_path = PATH['src_train']
pic_path = PATH['pic_train']
label_path = PATH['label_train']
csv_path = PATH['annotations_train']
generated_path = PATH['generated_train']

In [ ]:
patients = load_train()
df_node = pd.read_csv(csv_path+"annotations.csv")
df_node["file"] = df_node["seriesuid"].map(lambda file_name: get_filename(patients, file_name))
df_node = df_node.dropna()

In [ ]:
Parallel(n_jobs=-1)(delayed(create_samples)(patient) for patient in tqdm(sorted(patients)))

In [ ]:
CUBE_IMGTYPE_SRC = "_i"
patient_index = glob.glob(label_path + "*_annos_pos.csv")

In [ ]:
Parallel(n_jobs=-1)(delayed(make_pos_annotation_images)(patient_index, csv_file) for patient_index, csv_file in tqdm(enumerate(patient_index)))

## Val

In [ ]:
data_path = PATH['src_val']
pic_path = PATH['pic_val']
label_path = PATH['label_val']
generated_path = PATH['generated_val']

In [ ]:
patients = load_train()

In [ ]:
Parallel(n_jobs=-1)(delayed(create_tests)(patient) for patient in tqdm(sorted(patients)))

## Test

In [ ]:
data_path = PATH['src_test']
pic_path = PATH['pic_test']
label_path = PATH['label_test']
generated_path = PATH['generated_test']

In [ ]:
patients = load_train()

In [ ]:
Parallel(n_jobs=-1)(delayed(create_tests)(patient) for patient in tqdm(sorted(patients)))